In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
from xgboost import XGBRegressor
from datetime import date
from flask import Flask, request, jsonify
import pickle
import joblib

In [4]:
df = pd.read_csv("dataset_S1_S2.csv")
df

,BUSINESS_UNIT,CALIBRATION_DATE,CALIBRATION_DATE_LOCAL,CALIBRATION_DATE_LOCAL_TS,CALIBRATION_DATE_UTC,CALIBRATION_DATE_UTC_TS,DIVISION,IPN_NAME,LDA_PARTNUMBER,LDA_SERIALNUMBER,...,S2,SOURCE_DATABASE,VEPM_MP,VEPM_TIME_TAKEN,VEPU_MP,VEPU_TIME_TAKEN,VEP_MP,VEP_TIME_TAKEN,_delta_uuid,kafka_timestamp
0,PS-DP,22-APR-25 09:42:15.000000 PM +05:30,2025-04-22 21:42:15,2025-04-22 21:42:15.000000,2025-04-22T16:12:15.000000000,2025-04-22 16:12:15.000000,PS,VE,5447,11,...,1.161,japauto_app.apac.bosch.com,87.35,56.426,85.06,12.171,97.96,17.936,0460423078:005591:540,2025-04-22 17:55:30.518000
1,PS-DP,22-APR-25 09:27:12.000000 PM +05:30,2025-04-22 21:27:12,2025-04-22 21:27:12.000000,2025-04-22T15:57:12.000000000,2025-04-22 15:57:12.000000,PS,VE,5382,19,...,1.756,japauto_app.apac.bosch.com,100.85,81.879,89.24,14.483,111.53,22.341,0460426505:004148:540,2025-04-22 17:55:30.518000
2,PS-DP,22-APR-25 08:34:46.000000 PM +05:30,2025-04-22 20:34:46,2025-04-22 20:34:46.000000,2025-04-22T15:04:46.000000000,2025-04-22 15:04:46.000000,PS,VE,5382,2,...,1.773,japauto_app.apac.bosch.com,100.83,70.573,89.07,11.965,111.79,27.865,0460426505:004149:540,2025-04-22 16:55:30.140000
3,PS-DP,22-APR-25 05:42:12.000000 PM +05:30,2025-04-22 17:42:12,2025-04-22 17:42:12.000000,2025-04-22T12:12:12.000000000,2025-04-22 12:12:12.000000,PS,VE,5483,5,...,0.907,japauto_app.apac.bosch.com,88.63,48.563,80.49,15.853,92.00,21.881,0460424580:006085:540,2025-04-22 13:55:28.903000
4,PS-DP,22-APR-25 05:28:33.000000 PM +05:30,2025-04-22 17:28:33,2025-04-22 17:28:33.000000,2025-04-22T11:58:33.000000000,2025-04-22 11:58:33.000000,PS,VE,5483,1,...,0.974,japauto_app.apac.bosch.com,88.18,50.483,80.82,9.266,92.39,23.246,0460424580:006084:540,2025-04-22 13:55:28.902000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5559,PS-DP,25-SEP-23 09:11:37.000000 PM +05:30,2023-09-25 21:11:37,2023-09-25 21:11:37.000000,2023-09-25T15:41:37.000000000,2023-09-25 15:41:37.000000,PS,VE,5468,17,...,2.117,japauto_app.apac.bosch.com,94.80,36.892,69.43,24.599,99.25,22.314,0460424559:005448:377,2025-03-19 23:38:04.675000
5560,PS-DP,25-SEP-23 08:17:19.000000 PM +05:30,2023-09-25 20:17:19,2023-09-25 20:17:19.000000,2023-09-25T14:47:19.000000000,2023-09-25 14:47:19.000000,PS,VE,5468,2,...,1.978,japauto_app.apac.bosch.com,94.84,26.027,69.65,27.005,99.66,25.603,0460424559:005440:377,2025-03-19 23:38:04.702000
5561,PS-DP,25-SEP-23 06:32:05.000000 PM +05:30,2023-09-25 18:32:05,2023-09-25 18:32:05.000000,2023-09-25T13:02:05.000000000,2023-09-25 13:02:05.000000,PS,VE,5468,10,...,2.147,japauto_app.apac.bosch.com,94.71,51.589,69.60,16.197,99.79,17.907,0460424559:005446:377,2025-03-19 23:38:04.701000
5562,PS-DP,25-SEP-23 06:03:34.000000 PM +05:30,2023-09-25 18:03:34,2023-09-25 18:03:34.000000,2023-09-25T12:33:34.000000000,2023-09-25 12:33:34.000000,PS,VE,5468,13,...,2.179,japauto_app.apac.bosch.com,94.51,68.790,69.24,17.955,99.17,15.426,0460424559:005447:377,2025-03-19 23:38:04.702000


In [5]:
df = df.fillna(df.median(numeric_only = True))

## Outlier deletion

In [6]:
selected_features = ["S1","S2","VEPM_MP","VEPU_MP","VEP_MP","VEPM_TIME_TAKEN","VEPU_TIME_TAKEN","VEP_TIME_TAKEN"]
Q1 = df[selected_features].quantile(0.25)
Q3 = df[selected_features].quantile(0.75)
IQR = Q3 - Q1
df_filtered = df[~((df[selected_features] < (Q1 - 1.5 * IQR)) | 
                    (df[selected_features] > (Q3 + 1.5 * IQR))).any(axis=1)]
df = df_filtered
df

,BUSINESS_UNIT,CALIBRATION_DATE,CALIBRATION_DATE_LOCAL,CALIBRATION_DATE_LOCAL_TS,CALIBRATION_DATE_UTC,CALIBRATION_DATE_UTC_TS,DIVISION,IPN_NAME,LDA_PARTNUMBER,LDA_SERIALNUMBER,...,S2,SOURCE_DATABASE,VEPM_MP,VEPM_TIME_TAKEN,VEPU_MP,VEPU_TIME_TAKEN,VEP_MP,VEP_TIME_TAKEN,_delta_uuid,kafka_timestamp
0,PS-DP,22-APR-25 09:42:15.000000 PM +05:30,2025-04-22 21:42:15,2025-04-22 21:42:15.000000,2025-04-22T16:12:15.000000000,2025-04-22 16:12:15.000000,PS,VE,5447,11,...,1.161,japauto_app.apac.bosch.com,87.35,56.426,85.06,12.171,97.96,17.936,0460423078:005591:540,2025-04-22 17:55:30.518000
1,PS-DP,22-APR-25 09:27:12.000000 PM +05:30,2025-04-22 21:27:12,2025-04-22 21:27:12.000000,2025-04-22T15:57:12.000000000,2025-04-22 15:57:12.000000,PS,VE,5382,19,...,1.756,japauto_app.apac.bosch.com,100.85,81.879,89.24,14.483,111.53,22.341,0460426505:004148:540,2025-04-22 17:55:30.518000
2,PS-DP,22-APR-25 08:34:46.000000 PM +05:30,2025-04-22 20:34:46,2025-04-22 20:34:46.000000,2025-04-22T15:04:46.000000000,2025-04-22 15:04:46.000000,PS,VE,5382,2,...,1.773,japauto_app.apac.bosch.com,100.83,70.573,89.07,11.965,111.79,27.865,0460426505:004149:540,2025-04-22 16:55:30.140000
3,PS-DP,22-APR-25 05:42:12.000000 PM +05:30,2025-04-22 17:42:12,2025-04-22 17:42:12.000000,2025-04-22T12:12:12.000000000,2025-04-22 12:12:12.000000,PS,VE,5483,5,...,0.907,japauto_app.apac.bosch.com,88.63,48.563,80.49,15.853,92.00,21.881,0460424580:006085:540,2025-04-22 13:55:28.903000
4,PS-DP,22-APR-25 05:28:33.000000 PM +05:30,2025-04-22 17:28:33,2025-04-22 17:28:33.000000,2025-04-22T11:58:33.000000000,2025-04-22 11:58:33.000000,PS,VE,5483,1,...,0.974,japauto_app.apac.bosch.com,88.18,50.483,80.82,9.266,92.39,23.246,0460424580:006084:540,2025-04-22 13:55:28.902000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5559,PS-DP,25-SEP-23 09:11:37.000000 PM +05:30,2023-09-25 21:11:37,2023-09-25 21:11:37.000000,2023-09-25T15:41:37.000000000,2023-09-25 15:41:37.000000,PS,VE,5468,17,...,2.117,japauto_app.apac.bosch.com,94.80,36.892,69.43,24.599,99.25,22.314,0460424559:005448:377,2025-03-19 23:38:04.675000
5560,PS-DP,25-SEP-23 08:17:19.000000 PM +05:30,2023-09-25 20:17:19,2023-09-25 20:17:19.000000,2023-09-25T14:47:19.000000000,2023-09-25 14:47:19.000000,PS,VE,5468,2,...,1.978,japauto_app.apac.bosch.com,94.84,26.027,69.65,27.005,99.66,25.603,0460424559:005440:377,2025-03-19 23:38:04.702000
5561,PS-DP,25-SEP-23 06:32:05.000000 PM +05:30,2023-09-25 18:32:05,2023-09-25 18:32:05.000000,2023-09-25T13:02:05.000000000,2023-09-25 13:02:05.000000,PS,VE,5468,10,...,2.147,japauto_app.apac.bosch.com,94.71,51.589,69.60,16.197,99.79,17.907,0460424559:005446:377,2025-03-19 23:38:04.701000
5562,PS-DP,25-SEP-23 06:03:34.000000 PM +05:30,2023-09-25 18:03:34,2023-09-25 18:03:34.000000,2023-09-25T12:33:34.000000000,2023-09-25 12:33:34.000000,PS,VE,5468,13,...,2.179,japauto_app.apac.bosch.com,94.51,68.790,69.24,17.955,99.17,15.426,0460424559:005447:377,2025-03-19 23:38:04.702000


In [7]:
X = df[["LDA_PARTNUMBER","VEPM_MP","VEPM_TIME_TAKEN","VEPU_MP","VEPU_TIME_TAKEN","VEP_MP","VEP_TIME_TAKEN"]]
y = df[["S1","S2"]] 
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size = 0.2,random_state = 142)
print(X_train.shape,X_test.shape)
y_train.shape,y_test.shape

(4075, 7) (1019, 7)


((4075, 2), (1019, 2))

## Random Forest and  XGBoost Model

In [8]:
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

rf = RandomForestRegressor(n_estimators=100, max_depth=10, random_state=42)

cv_scores = cross_val_score(rf, X_train_scaled, y_train, cv=5, scoring="r2")

print(f"Cross-Validation R² Scores: {cv_scores}")
print(f"Mean R² Score: {cv_scores.mean()*100:.2f}%")

rf.fit(X_train_scaled, y_train)

y_pred_rf = rf.predict(X_test_scaled)

mse_rf = mean_squared_error(y_test, y_pred_rf)
r2_rf = r2_score(y_test, y_pred_rf)

print(f"Random Forest - MSE: {(mse_rf)*100:.2f}, R²: {(r2_rf)*100:.2f}%")

Cross-Validation R² Scores: [0.84458467 0.90111532 0.91531784 0.87470707 0.87020284]
Mean R² Score: 88.12%
Random Forest - MSE: 0.84, R²: 90.95%


In [9]:
scaler = RobustScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
xgb = XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)

cv_scores1 = cross_val_score(rf, X_train_scaled, y_train, cv=5, scoring="r2")

print(f"Cross-Validation R² Scores: {cv_scores1}")
print(f"Mean R² Score: {cv_scores1.mean()*100:.2f}%")

xgb.fit(X_train_scaled, y_train)

y_pred_xgb = xgb.predict(X_test_scaled)

mse_xgb = mean_squared_error(y_test, y_pred_xgb)
r2_xgb = r2_score(y_test, y_pred_xgb)

print(f"XGBoost - MSE: {(mse_xgb)*100:.2f}, R²: {(r2_xgb)*100:.2f}%")

Cross-Validation R² Scores: [0.84458467 0.90111532 0.91531784 0.87470707 0.87020284]
Mean R² Score: 88.12%
XGBoost - MSE: 0.75, R²: 91.47%


In [36]:
#              [LDA_partnumber,vepm,vepm_time,vepu,vepu_time,vep,vep_time]
manual_input = np.array([[5120, 75, 0, 62, 0, 82, 0]])

manual_scaled = scaler.transform(manual_input)

predicted_manual = xgb.predict(manual_scaled)

predicted = np.array(predicted_manual)

prediction_table = {
    "date": date.today(),
    "LDA_PARTNUMBER": manual_input[0][0],          
    "Predicted S1": predicted[0][0],
    "Predicted S2": predicted[0][1]
}

prediction_df = pd.DataFrame([prediction_table])

prediction_df


,date,LDA_PARTNUMBER,Predicted S1,Predicted S2
0,2025-06-19,5120,0.611602,1.255401


In [9]:
# import pickle

# try:
#     with open("xgb.pkl", "wb") as file:
#         pickle.dump(xgb, file)
#     print("Dataset saved successfully!")
# except Exception as e:
#     print(f"Error saving dataset: {e}")

# import joblib
# joblib.dump(model, "rf.pkl")
# print("Model saved successfully as rf.pkl")

In [11]:
# xgb.save_model("xgb_model.json")
# joblib.dump(scaler, "scaler.pkl")

['scaler.pkl']

In [35]:
model = XGBRegressor()
model.load_model("xgb_model.json")

# Sample test input
test_input = [[5120, 75, 0, 62, 0, 82, 0]]  # Replace with actual test data

# Make predictions
prediction = model.predict(test_input)
print("Prediction S1:", prediction[0:,0])
print("Prediction S2:", prediction[0:,1])

Prediction S1: [0.7387921]
Prediction S2: [1.6982199]
